In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from gpmap.utils import hamming_distance

from latticeproteins.conformations import Conformations
from latticeproteins.evolution import Evolution, AdaptiveWalk
from latticeproteins.fitness import Fitness
from latticeproteins.sequences import RandomSequence

from latticegpm.gpm import LatticeGenotypePhenotypeMap
from gpmap.evolve import monte_carlo, EvolverError
from gpmap.evolve.models import fixation

from epistasis.models.nonlinear.regression import EpistasisNonlinearRegression
from epistasis.models.linear.regression import EpistasisLinearRegression
from epistasis.models.nonlinear.power import EpistasisPowerTransform

In [2]:
length = 15
temperature = 1.0
dGdependence= "fracfolded"
c = Conformations(length)

In [3]:
def evolve():
    good = False
    while good is False:
        wildtype = "".join(RandomSequence(length))
        popsize = 50
        numsteps = 17
        f = None
        output = c.FoldSequence(wildtype, temperature)
        fitness = Fitness(temperature, c, dGdependence=dGdependence)
        finalpop = AdaptiveWalk(wildtype, fitness, numsteps, file=f, steepest_ascent=True)
        mutant = finalpop[0]
        order = hamming_distance(wildtype, mutant)
        if order == 13:
            return wildtype, mutant


In [6]:
with open("sequences.txt", "a") as f:
    counter = 0
    while counter < 10:
        wt, mut = evolve()
        print(counter)
        f.write(wt + ", " + mut + "\n")
        counter += 1
f.close()

0
1
2
3
4
5
6
7
8
9


In [14]:
length = 15
c = Conformations(length)
temperature = 1.0
good = False
while good is False:
    wildtype = "".join(RandomSequence(length))
    popsize = 20
    numsteps = 5
    f = None
    output = c.FoldSequence(wildtype, temperature)
    fitness = Fitness(temperature, c, dGdependence=dGdependence)
    finalpop = AdaptiveWalk(wildtype, fitness, numsteps, file=f, steepest_ascent=True)
    mutant = finalpop[0]
    order = hamming_distance(wildtype, mutant)
    if order == 5:
        print( wildtype, mutant )


VALNYPMAGNLYDVQ VAKNYKMVGNLFFVQ
SGAFDDLDETYTREP FGAFDMLWMKYTREP
CKEFHPSCQQGNQCD CKEFKPKFQQCNQCM
AEPWTVHERATVQDQ MEPWTVHIRAKVQHI
TIPMMRYIMRFMFRY TIPMKRKCMFFMFRC


KeyboardInterrupt: 